In [ ]:
import torch
import torch.nn as nn
from networks import *
import torch
import torch.nn as nn
import os
from networks import get_model
from datasets import data_merge
from optimizers import get_optimizer
from torch.utils.data import DataLoader
from transformers import *
from utils import *
from configs import parse_args
import time
import numpy as np
import random
import sys
from loss import *

pth_path = ''
model = get_model(4000)
model = nn.DataParallel(model)
model.load_state_dict(torch.load(pth_path)['state_dict'])
model.to('cuda')
model.eval()

data_path = ''
data_bank = data_merge(data_path)
dataset = data_bank.get_datasets(train=True, loo_domain="", img_size=256, transform=transformer_train_ImageNet())
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

all_domain = {}

dict_B = {
    '0': [],
    '1': []
}

all_domain['photo'] = dict_B
all_domain['replay_clear'] = dict_B
all_domain['replay_notclear'] = dict_B
all_domain['zalo'] = dict_B


with torch.no_grad():
    for i, sample_batched in enumerate(dataloader):
        image_x, label, UUID = sample_batched["image_x"].cuda(), sample_batched["label"], sample_batched["UUID"]
        cls_x1_x1, fea_x1_x1, fea_x1_x2 = model(image_x, image_x)
        all_domain[UUID][str(label)].append(fea_x1_x1.detach().cpu().numpy())